In [1]:
pip install transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
pip install -U fsspec datasets rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9d13ef40a726cf41d92054ce8cd931055d2e76c4064f98212761bef6bcf040f8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import torch
import os

# 1. Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# 2. Load dataset (small subset)
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset['train'].select(range(20))
val_data = dataset['validation'].select(range(10))

# 3. Load tokenizer and base model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 4. Preprocess function
max_input_length = 512
max_target_length = 128

def preprocess(example):
    input_text = "summarize: " + example["article"]
    target_text = example["highlights"]

    inputs = tokenizer(
        input_text, max_length=max_input_length, truncation=True, padding="max_length"
    )
    targets = tokenizer(
        target_text, max_length=max_target_length, truncation=True, padding="max_length"
    )

    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_data.map(preprocess, remove_columns=["article", "highlights", "id"])
val_dataset = val_data.map(preprocess, remove_columns=["article", "highlights", "id"])

# 5. Apply LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # Name may vary by model; for T5, use `q`, `v` inside attention
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, peft_config)

# 6. Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 7. Training arguments
training_args = TrainingArguments(
    output_dir="./t5-lora-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    learning_rate=5e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=1,
    fp16=torch.cuda.is_available()
)

# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 9. Train
trainer.train()

# 10. Save LoRA adapter only (efficient)
model.save_pretrained("./t5-lora-adapter")


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-5-1021954621.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,13.334726
2,No log,12.716680
3,No log,12.430014


In [6]:
text = "summarize: " + dataset['test'][0]['article']
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
print("Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))

TypeError: PeftModelForSeq2SeqLM.generate() takes 1 positional argument but 2 were given

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import PeftModel
import torch

# 1. Load base model and attach the LoRA adapter
base_model_name = "t5-small"
base_model = T5ForConditionalGeneration.from_pretrained(base_model_name)

# Path to LoRA adapter (trained & saved)
lora_model_path = "./t5-lora-adapter"

# Load LoRA-adapted model
model = PeftModel.from_pretrained(base_model, lora_model_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(base_model_name)

# 2. Function to summarize an article
def summarize_article(article_text):
    input_text = "summarize: " + article_text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Move inputs to the same device as the model
    if torch.cuda.is_available():
        inputs = inputs.to(model.device)

    # Generate summary
    summary_ids = model.generate(
        input_ids=inputs, # Explicitly pass input_ids as a keyword argument
        max_length=128,
        min_length=30,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 3. Example test article
test_article = """
Apple Inc. unveiled its latest line of iPhones during a virtual event on Tuesday.
The new models include the iPhone 14, iPhone 14 Pro, and iPhone 14 Pro Max, featuring improved cameras and longer battery life.
CEO Tim Cook emphasized Apple's focus on sustainability and innovation.
"""

# 4. Generate summary
print("Generated Summary:\n", summarize_article(test_article))

Generated Summary:
 the new models include the iPhone 14, iPhone 14 Pro, and iPhone 14 Pro Max. the new models feature improved cameras and longer battery life.


In [19]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from peft import PeftModel

base_model_name = "t5-small"
base_model = T5ForConditionalGeneration.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, "./t5-lora-adapter")

tokenizer = T5Tokenizer.from_pretrained(base_model_name)

text = "summarize: " + dataset['test'][0]['article']
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = model.base_model.generate(
    inputs["input_ids"],
    max_length=128,
    num_beams=4,
    early_stopping=True
)

print("Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


Summary:
 the Palestinian Authority officially became the 123rd member of the international criminal court. the ICC opened a preliminary examination into the situation in palestinians. the ICC also accepted its jurisdiction over alleged crimes committed in the occupied territories.


In [21]:
text = "summarize: " + dataset['test'][0]['article']
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = model.generate(input_ids=inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)

print("Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


Summary:
 the Palestinian Authority officially became the 123rd member of the international criminal court. the ICC opened a preliminary examination into the situation in palestinians. the ICC also accepted its jurisdiction over alleged crimes committed in the occupied territories.
